In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# 0) prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [3]:
n_samples, n_features = X.shape

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)

In [10]:
# scale the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# converting numpy data to tensor
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))

y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# reshaping the data
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [12]:
# 1) Model
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_features, 1)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred
    
model = Model(n_features)

In [13]:
# 2) Loss and optimizer

num_iters = 100
learning_rate = 0.01

criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [15]:
# 3) Training loop

for epochs in range(num_iters):
    # forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    
    # backward pass and update
    loss.backward()
    optimizer.step()
    
    # Zero grad before new step
    optimizer.zero_grad()
    
    if (epochs+1) % 10 == 0:
        print(f'epoch: {epochs+1}, loss = {loss.item():.4f}')
        
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

epoch: 10, loss = 0.4007
epoch: 20, loss = 0.3574
epoch: 30, loss = 0.3254
epoch: 40, loss = 0.3006
epoch: 50, loss = 0.2808
epoch: 60, loss = 0.2644
epoch: 70, loss = 0.2507
epoch: 80, loss = 0.2389
epoch: 90, loss = 0.2287
epoch: 100, loss = 0.2197
accuracy: 0.9211
